In [21]:
import time
from datetime import datetime as dt

from google.cloud import bigtable
from google.cloud.bigtable import column_family, row_filters
from google.cloud.bigtable.row_set import RowSet
import google.cloud.bigtable.row_filters as row_filters
import pandas as pd



In [9]:
INSTANCE_ID = 'bigtable-1'
TABLE_ID = 'plant2Temperatures_{}'.format(time.time())

In [10]:
TABLE_ID

'plant2Temperatures_1649627962.5933096'

### Utils functions

In [23]:
def print_row(row):
    print("Reading data for {}:".format(row.row_key.decode('utf-8')))
    for cf, cols in sorted(row.cells.items()):
        print("Column Family {}".format(cf))
        for col, cells in sorted(cols.items()):
            for cell in cells:
                labels = " [{}]".format(",".join(cell.labels)) \
                    if len(cell.labels) else ""
                print(
                    "\t{}: {} @{}{}".format(col.decode('utf-8'),
                                            cell.value.decode('utf-8'),
                                            cell.timestamp, labels))
    print("")

### Connecting to BigTable

In [11]:
client = bigtable.Client.from_service_account_json('./json/unbosque-service-account.json', admin = True)

In [12]:
print('Connecting to database instance')
instance = client.instance(INSTANCE_ID)

Connecting to database instance


### Creating a table

In [13]:
print('Creating the {} table'.format(TABLE_ID))
table = instance.table(TABLE_ID)

if not table.exists():
    table.create()
else:
    print("ERROR: Table {} already exists".format(TABLE_ID))

Creating the plant2Temperatures_1649627962.5933096 table


In [14]:
print('Creating all family columns')

plantId = 'Temperature_sensor'
plant_cf = table.column_family(plantId)
plant_cf.create()


Creating all family columns


### Inserting data

In [17]:

p2_weatherdata = pd.read_csv("../data/Plant_2_Weather_Sensor_Data.csv")
p2_json = p2_weatherdata.to_dict(orient='records')

In [25]:
dt = dt.utcnow()
rows = []

print('Writing orders to the table')
for temperature in p2_json:
    
    row_key = 'plant2#{}'.format(temperature['DATE_TIME']).encode()
    row = table.direct_row(row_key)
    
    row.set_cell(plantId, 'temperatura_ambiente'.encode(), str(temperature['AMBIENT_TEMPERATURE']), timestamp = dt)
    row.set_cell(plantId, 'Modulo_temperatura'.encode(), str(temperature['MODULE_TEMPERATURE']), timestamp = dt)
    row.set_cell(plantId, 'radiacion'.encode(), str(temperature['IRRADIATION']), timestamp = dt)
    
    
    
    rows.append(row)

table.mutate_rows(rows)

Writing orders to the table


[,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 

### Reading data

In [26]:
def read_row_range():

    row_set = RowSet()
    row_set.add_row_range_from_keys(
        start_key=b"plant2#2020-06-01 06:00:00", end_key=b"plant2#2020-06-01 12:00:00"
    )

    filter2=row_filters.ColumnQualifierRegexFilter(b"temperatura_ambiente")
   
    rows = table.read_rows(row_set=row_set,filter_=filter2)
    for row in rows:
        print_row(row)

read_row_range()


Reading data for plant2#2020-06-01 06:00:00:
Column Family Temperature_sensor
	temperatura_ambiente: 23.470155033333327 @2022-04-10 23:27:27.616000+00:00
	temperatura_ambiente: 23.470155033333327 @2022-04-10 22:01:51.940000+00:00

Reading data for plant2#2020-06-01 06:15:00:
Column Family Temperature_sensor
	temperatura_ambiente: 23.67332510344828 @2022-04-10 23:27:27.616000+00:00
	temperatura_ambiente: 23.67332510344828 @2022-04-10 22:01:51.940000+00:00

Reading data for plant2#2020-06-01 06:30:00:
Column Family Temperature_sensor
	temperatura_ambiente: 23.979217633333327 @2022-04-10 23:27:27.616000+00:00
	temperatura_ambiente: 23.979217633333327 @2022-04-10 22:01:51.940000+00:00

Reading data for plant2#2020-06-01 06:45:00:
Column Family Temperature_sensor
	temperatura_ambiente: 24.209690400000003 @2022-04-10 23:27:27.616000+00:00
	temperatura_ambiente: 24.209690400000003 @2022-04-10 22:01:51.940000+00:00

Reading data for plant2#2020-06-01 07:00:00:
Column Family Temperature_sensor
